### Procédure de test et fonctionnement de deviation.py (Objectif 2)

**1. Tests des Fonctions Utilitaires**

| ID  | Fonctionnalité                 | Procédure                                               | Données de test                      | Résultat attendu                                                                       |
| :-- | :----------------------------- | :------------------------------------------------------ | :----------------------------------- | :------------------------------------------------------------------------------------- |
| 1  | `champ_electrique_v2` valide   | Appeler `champ_electrique_v2(0.1, 5000)`                | distance=0.1 m, potentiel=5000 V   | Retourne 50000.0.                                                                      |
| 2  | `champ_electrique_v2` dist=0   | Appeler `champ_electrique_v2(0, 5000)`                  | distance=0 m, potentiel=5000 V     | Lève une `ValueError` avec message "La distance doit être positive.".                 |
| 3  | `champ_electrique_v2` dist<0   | Appeler `champ_electrique_v2(-0.1, 5000)`               | distance=-0.1 m, potentiel=5000 V  | Lève une `ValueError` avec message "La distance doit être positive.".                 |

*Note : La fonction `calcul_champ_electrique` n'est pas présente dans Objectif 2.*

**2. Tests de la Classe `particule` (Initialisation et Calculs)**

| ID  | Fonctionnalité                     | Procédure                                                                    | Données de test                                                     | Résultat attendu                                                                                                                               |
| :-- | :--------------------------------- | :--------------------------------------------------------------------------- | :------------------------------------------------------------------ | :--------------------------------------------------------------------------------------------------------------------------------------------- |
| 1  | Initialisation valide | `p = particule((1, 1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)` | m=1u, c=1e, v0=1e5, angle=pi/4, h=0.1                             | Objet `p` créé. `p.m=1`, `p.c=1`, `p.vo=1e5`, `p.angle=np.pi/4`, `p.height=0.1`. `p.mq` calculé (approx 1.036e-8 kg/C). `is_incertitude=False`. |
| 2  | Initialisation charge nulle        | Tenter `p = particule((1, 0))`                                                 | m=1u, c=0e                                                          | Lève une `ValueError` avec message "La charge de la particule ne peut pas être nulle.".  |
| 3  | Initialisation incertitude       | `p = particule((1.1, 0.9), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1, is_incertitude=True, incertitude_unique=True, base_mq=(1,1))` | m=1.1u, c=0.9e, v0, angle, h, is_incertitude=True, unique=True, base=(1,1) | Objet `p` créé. `p.is_incertitude=True`, `p.incertitude_unique=True`, `p.base_mq=(1,1)`.                                                     |
| 4  | `equation_trajectoire` (cas simple)| `p = particule((1, 1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)`. Calculer `p.equation_trajectoire(0.01, -50000)` | x=0.01m, E=-50kV/m (donné)             | Retourne une valeur `y` float (calcul analytique: y(0.01) = 0.1 - 0.01*cot(pi/4) + 0.5*(-50k/p.mq)*(0.01/(1e5*sin(pi/4)))^2 ≈ 0.0851 m).           |
| 5  | `trajectoire`                    | `p = particule((1,1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)`. Calculer `x, y = p.trajectoire(-50000, 0, 0.1, n_points=100)` | E=-50kV/m, x_min=0, x_max=0.1, n=100                                      | Retourne deux `np.ndarray` `x` et `y`, chacun de taille 100. `x` va de 0 à 0.1.                                                                 |
| 6  | `point_contact` (contact attendu)  | `p = particule((1, 1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)`. Calculer `p.point_contact(-50000)` | E = -50000 V/m                                                  | Retourne une valeur `x` float positive (approx. 0.0618 m).                                                                       |
| P7  | `point_contact` (pas de contact)   | `p = particule((1, 1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)`. Calculer `p.point_contact(1000)`   | E = +1000 V/m (champ déviant vers +y)                                  | Retourne `None` (car `(v0*cos(angle))^2 - 2*h*E/mq < 0`).                                                                                                   |
| 8  | `point_contact` (E=0)              | `p = particule((1, 1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)`. Calculer `p.point_contact(0)`    | E = 0 V/m                                                         | Retourne `h * tan(angle)` = 0.1 * tan(pi/4) = 0.1 m.                                                                                            |
| 9  | `angle_incident` (contact attendu) | `p = particule((1, 1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)`. Calculer `p.angle_incident(-50000)` | E = -50000 V/m                                                  | Retourne un angle float en radians (approx. 1.066 rad ou 61.1° vs +y).                                                                        |
| 10 | `angle_incident` (pas de contact)  | `p = particule((1, 1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)`. Calculer `p.angle_incident(1000)`  | E = +1000 V/m                                                   | Lève une `TypeError` (car `x_contact` est `None` et on tente une opération dessus). *Correction : devrait retourner `None` si `x_contact` est `None`.* |

**3. Tests des Fonctions de Tracé (Statique - Multiples Particules)**

| ID  | Fonctionnalité                             | Procédure                                                                                                  | Données de test                                                            | Résultat attendu (si `create_plot=False`)                                                                                               |
| :-- | :----------------------------------------- | :--------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------- |
| 1  | `tracer_ensemble_trajectoires` valide      | `fig, ax = plt.subplots(); tracer_ensemble_trajectoires([(1,1),(2,1)], 1e5, potentiel=5000, hauteur_initiale=0.1, ax=ax, create_plot=False)` | Particules (1,1), (2,1), V0=1e5, Pot=5000, h=0.1                               | L'objet `ax` est modifié : contient 2 lignes (trajectoires), 1 ligne (échantillon), légende, texte des angles. Retourne `ax`. Pas d'erreur. |
| 2  | `tracer_ensemble_trajectoires` charge incompatible | Tenter `tracer_ensemble_trajectoires([(1,1),(2,-1)], 1e5, ...)`                                                    | Charges +1 et -1                                                           | Lève `ValueError` avec message "Les charges de toutes les particules doivent être du même signe et être non nulle".                 |
| 3  | `tracer_ensemble_trajectoires` pas contact | `fig, ax = plt.subplots(); tracer_ensemble_trajectoires([(1,1)], 1e5, potentiel=-100, hauteur_initiale=0.1, ax=ax, create_plot=False)`                                    | Pot=-100 (faible, déviation vers +y si q>0, mais peut ne pas toucher)                                                        | `ax` modifié. Trajectoire tracée jusqu'à la limite x du graphe. Texte indique "Pas de contact".                                      |
| 4  | `create_particules_incertitudes` (qE>=0)    | `p=particule((1,1),v_initiale=1e5,hauteur_initiale=0.1); inc={'m':0.1,'q':0.1,'v0':0.1,'theta':0.1,'h':0.1,'E':0.1}; E_val = 50000; create_particules_incertitudes([p], inc, E_val)`          | Particule (1,1), E>0 (qE>0), incertitudes 10%                                   | Retourne `(final_particules, E_min, E_max)`. `final_particules` contient `[p, min_p, max_p]`. `min_p` a `m*(1+inc_m)`, `q*(1-inc_q)`, etc. `E_min = E*(1-inc_E)`. |
| 5  | `create_particules_incertitudes` (qE<0)    | `p=particule((1,1),v_initiale=1e5,hauteur_initiale=0.1); inc={'m':0.1,'q':0.1,'v0':0.1,'theta':0.1,'h':0.1,'E':0.1}; E_val = -50000; create_particules_incertitudes([p], inc, E_val)`                       | Particule (1,1), E<0 (qE<0), incertitudes 10%                                            | Retourne `(final_particules, E_min, E_max)`. `min_p` a `m*(1-inc_m)`, `q*(1+inc_q)`, etc. `E_min = E*(1+inc_E)`.                 |
| 6  | `tracer_ensemble_trajectoires_avec_incertitudes` valide | `fig,ax=plt.subplots(); inc_test={'m':0.01,'q':0.01,'v0':0.01,'theta':0.01,'h':0.01,'E':0.01}; tracer_ensemble_trajectoires_avec_incertitudes([(1,1)], 1e5, inc_test, potentiel=5000, hauteur_initiale=0.1, ax=ax, create_plot=False)`                   | Particule (1,1), incertitudes valides, Pot=5000, h=0.1                           | `ax` modifié. Contient 1 ligne pleine (nominal), 2 lignes pointillées (incertitudes). Couleurs cohérentes. Label incertitude présent.       |
| 7  | `tracer_ensemble_trajectoires_avec_incertitudes` charge incompatible| Tenter `tracer_ensemble_trajectoires_avec_incertitudes([(1,1),(2,-1)], 1e5, {}, ...)`                                                                 | Charges +1 et -1                                                           | Lève `ValueError` avec message "Les charges de toutes les particules doivent être du même signe et être non nulle".                 |

**4. Tests des Fonctions de Tracé (Statique - Une Particule, Multiples Potentiels)**

| ID  | Fonctionnalité                                          | Procédure                                                                                                      | Données de test                                                             | Résultat attendu (si `create_plot=False`)                                                                                             |
| :-- | :------------------------------------------------------ | :------------------------------------------------------------------------------------------------------------- | :-------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------ |
| 1 | tracer_ensemble_potentiels valide                     | fig,ax=plt.subplots(); tracer_ensemble_potentiels((1,1), 1e5, potentiels=[5000, 10000], hauteur_initiale=0.1, ax=ax, create_plot=False)           | Particule (1,1), V0=1e5, Pots=[5k, 10k], h=0.1                               | `ax` modifié. Contient 2 lignes (une par potentiel), 1 ligne échantillon, légende avec labels "Trajectoire à 5000.0 V", etc. Texte angles OK. |
| 2 | tracer_ensemble_potentiels charge nulle               | Tenter `tracer_ensemble_potentiels((1,0), 1e5, ...)`                                                                  | Charge=0                                                                    | Lève `ValueError` (levée par le constructeur `particule`).                                                                          |
| 3 | tracer_ensemble_trajectoires_potentiels_avec_incertitudes valide   | fig,ax=plt.subplots(); inc_test={'m':0.01,'q':0.01,'v0':0.01,'theta':0.01,'h':0.01,'E':0.01}; tracer_ensemble_trajectoires_potentiels_avec_incertitudes((1,1), 1e5, inc_test, potentiels=[5000, 10000], hauteur_initiale=0.1, ax=ax, create_plot=False)                    | Particule (1,1), incertitudes, Pots=[5k, 10k], h=0.1                          | `ax` modifié. Contient 2*3=6 lignes (1 nominale + 2 incertitudes par potentiel). Labels et couleurs corrects. Texte angles OK.             |
| 4 | tracer_ensemble_trajectoires_potentiels_avec_incertitudes charge nulle| Tenter `tracer_ensemble_trajectoires_potentiels_avec_incertitudes((1,0), 1e5, {}, ...)`                                                                               | Charge=0                                                                    | Lève `ValueError` (levée par le constructeur `particule`).                                                                          |
